In [1]:
import datetime as dt
import numpy as np
import spiceypy as sp
import plotly.express as px

sp.furnsh('kernel_meta.txt')

In [2]:
initial_time_utc = dt.datetime(year=2000, month=1, day=1, hour=0, second=0)
delta_days = 10000
final_time_utc = initial_time_utc + dt.timedelta(days=delta_days)

In [3]:
initial_time_utc_str = initial_time_utc.strftime('%Y-%m-%dT%H:%M:%S')
final_time_utc_str = final_time_utc.strftime('%Y-%m-%dT%H:%M:%S')
print(initial_time_utc_str)
print(final_time_utc_str)

2000-01-01T00:00:00
2027-05-19T00:00:00


In [4]:
initial_time_et = sp.utc2et(initial_time_utc_str)
final_time_et = sp.utc2et(final_time_utc_str)
print(initial_time_et)
print(final_time_et)

-43135.816087188054
863956869.1851972


In [12]:
# Solar System Barycenter with respect to the Sun
time_interval_et = np.linspace(start=initial_time_et, stop=final_time_et, num=delta_days)
print(time_interval_et)
print(len(time_interval_et))

ssb_wrt_sun_position = []
for time_inteval in time_interval_et:
    # _ is a convention to mark a temporary variable
    _position, _ = sp.spkgps(targ=0, et=time_inteval, ref='ECLIPJ2000', obs=10)
    ssb_wrt_sun_position.append(_position) 

ssb_wrt_sun_position = np.array(ssb_wrt_sun_position)

[-4.31358161e+04  4.32728253e+04  1.29681467e+05 ...  8.63784052e+08
  8.63870461e+08  8.63956869e+08]
10000


In [13]:
_, radii_sun = sp.bodvcd(bodyid=10, item='RADII', maxn=3)

radius_sun = radii_sun[0]

# Scale the position values using the Sun's radius
ssb_wrt_sun_position_scaled = ssb_wrt_sun_position / radius_sun

print(ssb_wrt_sun_position_scaled)
ssb_wrt_sun_position_scaled_xy = ssb_wrt_sun_position_scaled[:, 0:2]
print(ssb_wrt_sun_position_scaled_xy)

[[ 1.53448303  0.6001163  -0.04431704]
 [ 1.53332687  0.60170849 -0.04429676]
 [ 1.53216891  0.60329895 -0.04427644]
 ...
 [ 0.00475804  0.93720876 -0.01453411]
 [ 0.00378926  0.93632677 -0.01451128]
 [ 0.00282197  0.93544375 -0.01448847]]
[[1.53448303 0.6001163 ]
 [1.53332687 0.60170849]
 [1.53216891 0.60329895]
 ...
 [0.00475804 0.93720876]
 [0.00378926 0.93632677]
 [0.00282197 0.93544375]]


In [28]:
x_pos = ssb_wrt_sun_position_scaled[:, 0]
y_pos = ssb_wrt_sun_position_scaled[:, 1]

import plotly.graph_objects as go
# Create scatter trace of text labels
fig = px.scatter(x=x_pos, y=y_pos, template='plotly_dark')

# Add Sun
fig.add_shape(type='circle',
    xref='x', yref='y',
    x0=x_pos, y0=y_pos,
    line_color='Yellow',
    fillcolor='Yellow',
    layer='below'
)


fig.update_layout(width=400, height=400)
fig.update(layout_yaxis_range=[-2,2], layout_xaxis_range=[-2,2])
fig.show()